In [44]:
import pandas as pd

### Transposing the 2021 neighborhood data, and adding headers
The new df is saved to '../../data/processed/proposal_exploration/neighborhood_df2021.csv'

In [46]:
neighborhood_df = pd.read_excel('../../data/raw/neighbourhood-profiles-2021-158-model.xlsx')

In [47]:
#Transposing df (swapping rows and columns)
neighborhood_df = neighborhood_df.transpose()

In [49]:
# Set first row as column header in DF
neighborhood_df.columns = neighborhood_df.iloc[0]

# Remove the first row since it's now the column header
neighborhood_df2021 = neighborhood_df.iloc[1:]

neighborhood_df2021
neighborhood_df2021.to_csv('../../data/processed/proposal_exploration/neighborhood_df2021.csv')


### Adding the neighborhood number to each intersection
The new df is saved to ../../data/processed/proposal_exploration/intersectionsAndNeighborhoods.csv

In [66]:
import requests

locations_df = pd.read_csv('../../data/raw/traffic-volume-intersections/locations.csv')

locations_df = locations_df.reset_index()

locations_df

,index,_id,location_id,location,lng,lat,centreline_type,centreline_id,px,latest_count_date
0,0,1,1146,ELLESMERE RD AT PARKINGTON CRES,-79.246254,43.773319,2.0,13446642.0,2296.0,2022-12-17
1,1,2,1981,YORK MILLS RD E/B & W/B TO DON VALLEY PKWY N/B,-79.334658,43.757336,1.0,440171.0,NaN,2004-04-08
2,2,3,3468,PARKSIDE DR N/B S OF SPRING RD,-79.454442,43.640512,1.0,30010748.0,NaN,2021-12-15
3,3,4,3925,RIPLEY AVE AT SOUTH KINGSWAY,-79.475274,43.636780,2.0,13468657.0,NaN,2022-01-27
4,4,5,3926,BLOOR ST AT SOUTH KINGSWAY & RIVERVIEW GARDENS...,-79.485752,43.648312,2.0,13467247.0,334.0,2022-05-10
...,...,...,...,...,...,...,...,...,...,...
6066,6066,6067,50112,St Phillips Rd / Riverview Hts,-79.531895,43.703185,2.0,13457971.0,NaN,2023-11-28
6067,6067,6068,50113,Burbank Dr / Hawksbury Dr,-79.377618,43.774212,2.0,13446789.0,NaN,2023-11-28
6068,6068,6069,50114,Bathurst St / Hillhurst Blvd,-79.427281,43.708771,2.0,13456842.0,NaN,2023-11-28
6069,6069,6070,50116,Bayford Ave / Tilbury Dr / Ln S Cuffley W Tilbury,-79.480086,43.731976,2.0,13453346.0,NaN,2023-12-05


In [ ]:
def get_neighbourhood_id(long, lat):
    try:
        url = f'https://services3.arcgis.com/b9WvedVPoizGfvfD/arcgis/rest/services/COTGEO_NEIGHBOURHOOD_PROFILE/FeatureServer/0/query?returnGeometry=true&where=1%3D1&outSr=4326&outFields=*&inSr=4326&geometry=%7B%22xmin%22%3A{long}%2C%22ymin%22%3A{lat}%2C%22xmax%22%3A{long}%2C%22ymax%22%3A{lat}%2C%22spatialReference%22%3A%7B%22wkid%22%3A4326%7D%7D&geometryType=esriGeometryEnvelope&spatialRel=esriSpatialRelIntersects&geometryPrecision=6&f=geojson'
        print(f"API request for longitude: {long}, latitude: {lat}")
        response = requests.get(url,timeout=3)
        if response.status_code == 200:
            features = response.json().get('features', [])
            if features:
                print(f"Received API response {features} for longitude: {long}, latitude: {lat}")
                return features[0]['properties'].get('NEIGHBOURHOODID')
            else:
                return None
        else:
            return None
    except Exception as e:
        print(f"Error: {e}")
        return None
# resp = get_neighbourhood_id(-79.334658, 43.757336)
# print (resp)
    
# locations_df.tail(1)    
# df1 = locations_df.tail(1)  
    
# Add the neighborhood ID to each row. Runinng this will take an estimated 30 mins!
locations_df['neighbourhood_id'] = locations_df.apply(lambda row: get_neighbourhood_id(row['lng'], row['lat']), axis=1)

locations_df

locations_df.to_csv('../../data/processed/proposal_exploration/intersectionsAndNeighborhoods.csv')